In [0]:
# coding:utf-8
 
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import glob

In [0]:
from google.colab import drive
drive.mount('/content/drive')

###グラフプロット用関数

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def plt_result(epochs, history):
    plt.plot(range(1, epochs+1), history.history['acc'], label="training")
    plt.plot(range(1, epochs+1), history.history['val_acc'], label="validation")
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    predict_classes = model.predict_classes(X_test, batch_size=32)
    print_cmx(Y_test, predict_classes, TRAIN_DATA_FILES)

###ヒートマップ作製用関数

In [0]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
def print_cmx(y_true, predict, index=None):
    true_classes = np.argmax(y_true,1)

    cmx_data = confusion_matrix(true_classes, predict)

    df_cmx = pd.DataFrame(cmx_data, index=index, columns=index)

    plt.figure(figsize = (10,7))
    sns.heatmap(df_cmx, annot=True)
    plt.show()

### read data

In [0]:
folder = ["corner","cross","dead_end","straight", "3-way_junction"]
image_size = 50

X = []
Y = []
for index, name in enumerate(folder):
    dir = "/content/drive/My Drive/Colab Notebooks/data_junction/" + name
    files = glob.glob(dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("L")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        X.append(data)
        Y.append(index)

X = np.array(X)
Y = np.array(Y)

X = X.astype('float32') / 255.0
Y = np_utils.to_categorical(Y, len(folder))

###train_data, valid_data & test_data作成

In [0]:
X_train, X_valid_and_test, y_train, y_valid_and_test = train_test_split(X, Y, test_size=0.40)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid_and_test, y_valid_and_test, test_size=0.50)

###ネットワーク設定

In [0]:
model = Sequential()
 
model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.75))
 
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.75))
 
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(4))
model.add(Activation('softmax'))
 

model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])


###学習

In [0]:
history = model.fit(X_train, y_train, epochs=120)